In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
csv0 = pd.read_csv("csv0.txt")
csv1 = pd.read_csv("csv1.txt")
csv2 = pd.read_csv("csv2.txt")
csv3 = pd.read_csv("csv3.txt")
csv4 = pd.read_csv("csv4.txt")
csv5 = pd.read_csv("csv5.txt")

C:\Users\kpngb\AppData\Local\Temp\ipykernel_339792\3406777797.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv0 = pd.read_csv("csv0.txt")
C:\Users\kpngb\AppData\Local\Temp\ipykernel_339792\3406777797.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv2 = pd.read_csv("csv2.txt")
C:\Users\kpngb\AppData\Local\Temp\ipykernel_339792\3406777797.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv3 = pd.read_csv("csv3.txt")
C:\Users\kpngb\AppData\Local\Temp\ipykernel_339792\3406777797.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv4 = pd.read_csv("csv4.txt")


In [62]:
df = pd.concat([csv0, csv1, csv2, csv3, csv4, csv5], ignore_index=True)

In [63]:
df

,game_id,player_id,team,won,agent,rating,average_combat_score,kills,deaths,assists,kills_deaths,kill_assist_trade_survive_ratio,average_damage_per_round,headshot_ratio,first_kills,first_deaths,side,tier,rounds_won
0,179438,Add3r,DZ,False,Killjoy,0.73,245,10,11,1,-1.0,75%,151.0,45%,1.0,0.0,t,21,2
1,179438,Add3r,DZ,False,Killjoy,1.15,353,4,3,0,1.0,100%,204.0,38%,0.0,0.0,ct,21,0
2,179438,Rawkus,DZ,False,Breach,0.90,180,7,10,7,-3.0,75%,119.0,42%,1.0,1.0,t,21,2
3,179438,Rawkus,DZ,False,Breach,0.28,35,0,3,1,-3.0,67%,35.0,0%,0.0,0.0,ct,21,0
4,179438,corey,DZ,False,Neon,0.67,140,6,10,2,-4.0,50%,104.0,38%,1.0,3.0,t,21,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791175,16034,Retla,PRX,False,Reyna,0.78,251,10,11,1,-1.0,67%,121.0,9%,0.0,1.0,ct,21,5
791176,16034,lenne,PRX,False,Sova,0.58,104,4,9,3,-5.0,67%,68.0,6%,1.0,0.0,t,21,6
791177,16034,lenne,PRX,False,Sova,0.81,153,7,8,2,-1.0,33%,92.0,19%,0.0,3.0,ct,21,5
791178,16034,shiba,PRX,False,Brimstone,0.86,206,8,9,0,-1.0,42%,150.0,31%,3.0,1.0,t,21,6


In [64]:
df.isna().sum()

game_id                               0
player_id                             0
team                               1890
won                                   0
agent                                 0
rating                              170
average_combat_score                  0
kills                                 0
deaths                                0
assists                               0
kills_deaths                        472
kill_assist_trade_survive_ratio    1016
average_damage_per_round            454
headshot_ratio                     2724
first_kills                          20
first_deaths                         10
side                                  0
tier                                  0
rounds_won                            0
dtype: int64

In [65]:
df = df.dropna()

In [66]:
df.columns

Index(['game_id', 'player_id', 'team', 'won', 'agent', 'rating',
       'average_combat_score', 'kills', 'deaths', 'assists', 'kills_deaths',
       'kill_assist_trade_survive_ratio', 'average_damage_per_round',
       'headshot_ratio', 'first_kills', 'first_deaths', 'side', 'tier',
       'rounds_won'],
      dtype='object')

In [67]:
df = df.drop(["average_combat_score"], axis=1)

In [68]:
df['kill_assist_trade_survive_ratio'] = df['kill_assist_trade_survive_ratio'].str.replace('%', '').astype(float)
df['headshot_ratio'] = df['headshot_ratio'].str.replace('%', '').astype(float)

### Preprocessing

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [70]:
df_new = df.drop(["game_id", "player_id", "team"], axis=1)

In [71]:
df_new = pd.get_dummies(df_new)

In [72]:
df_new

,won,rating,kills,deaths,assists,kills_deaths,kill_assist_trade_survive_ratio,average_damage_per_round,headshot_ratio,first_kills,...,agent_Phoenix,agent_Raze,agent_Reyna,agent_Sage,agent_Skye,agent_Sova,agent_Viper,agent_Yoru,side_ct,side_t
0,False,0.73,10,11,1,-1.0,75.0,151.0,45.0,1.0,...,False,False,False,False,False,False,False,False,False,True
1,False,1.15,4,3,0,1.0,100.0,204.0,38.0,0.0,...,False,False,False,False,False,False,False,False,True,False
2,False,0.90,7,10,7,-3.0,75.0,119.0,42.0,1.0,...,False,False,False,False,False,False,False,False,False,True
3,False,0.28,0,3,1,-3.0,67.0,35.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
4,False,0.67,6,10,2,-4.0,50.0,104.0,38.0,1.0,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791175,False,0.78,10,11,1,-1.0,67.0,121.0,9.0,0.0,...,False,False,True,False,False,False,False,False,True,False
791176,False,0.58,4,9,3,-5.0,67.0,68.0,6.0,1.0,...,False,False,False,False,False,True,False,False,False,True
791177,False,0.81,7,8,2,-1.0,33.0,92.0,19.0,0.0,...,False,False,False,False,False,True,False,False,True,False
791178,False,0.86,8,9,0,-1.0,42.0,150.0,31.0,3.0,...,False,False,False,False,False,False,False,False,False,True


In [73]:
X = df_new.drop(columns=['won'])
y = df_new['won']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=81)

### Training Models

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

In [76]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

c:\Users\kpngb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [77]:
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

In [78]:
accuracy= accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_prob[:,1])
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("MSE:", mse)
print("Classification Report:\n", report)

Accuracy: 0.7755082549647581
MSE: 0.15314024161539372
Classification Report:
               precision    recall  f1-score   support

       False       0.77      0.77      0.77     78150
        True       0.78      0.78      0.78     78907

    accuracy                           0.78    157057
   macro avg       0.78      0.78      0.78    157057
weighted avg       0.78      0.78      0.78    157057



In [79]:
feature_names = X.columns
coefficients = model.coef_[0]
intercept = model.intercept_

coefs = pd.DataFrame({
    'Feature': feature_names,
    'Coefficients': coefficients
})

print(coefs)

                            Feature  Coefficients
0                            rating      1.913709
1                             kills     -0.183536
2                            deaths     -0.148849
3                           assists     -0.050827
4                      kills_deaths     -0.034687
5   kill_assist_trade_survive_ratio      0.011632
6          average_damage_per_round      0.001256
7                    headshot_ratio      0.001576
8                       first_kills      0.009012
9                      first_deaths      0.077870
10                             tier     -0.166639
11                       rounds_won      0.607648
12                      agent_Astra     -0.032340
13                     agent_Breach     -0.097321
14                  agent_Brimstone     -0.076924
15                    agent_Chamber     -0.035058
16                      agent_Clove      0.086876
17                     agent_Cypher     -0.023536
18                   agent_Deadlock     -0.012438


In [80]:
coefs.to_csv('coef.txt', sep=' ')